# rSLDS code

### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp

### NASCAR

In [ ]:
# Simulate the "nascar" data
def make_nascar_model():
    As = [random_rotation(D_latent, np.pi/24.),
      random_rotation(D_latent, np.pi/48.)]

    # Set the center points for each system
    centers = [np.array([+2.0, 0.]),
           np.array([-2.0, 0.])]
    bs = [-(A - np.eye(D_latent)).dot(center) for A, center in zip(As, centers)]

    # Add a "right" state
    As.append(np.eye(D_latent))
    bs.append(np.array([+0.1, 0.]))

    # Add a "right" state
    As.append(np.eye(D_latent))
    bs.append(np.array([-0.25, 0.]))

    # Construct multinomial regression to divvy up the space
    w1, b1 = np.array([+1.0, 0.0]), np.array([-2.0])   # x + b > 0 -> x > -b
    w2, b2 = np.array([-1.0, 0.0]), np.array([-2.0])   # -x + b > 0 -> x < b
    w3, b3 = np.array([0.0, +1.0]), np.array([0.0])    # y > 0
    w4, b4 = np.array([0.0, -1.0]), np.array([0.0])    # y < 0
    Rs = np.row_stack((100*w1, 100*w2, 10*w3,10*w4))
    r = np.concatenate((100*b1, 100*b2, 10*b3, 10*b4))
    
    true_rslds = ssm.SLDS(D_obs, K, D_latent, 
                      transitions="recurrent_only",
                      dynamics="diagonal_gaussian",
                      emissions="gaussian_orthog",
                      single_subspace=True)
    true_rslds.dynamics.mu_init = np.tile(np.array([[0, 1]]), (K, 1))
    true_rslds.dynamics.sigmasq_init = 1e-4 * np.ones((K, D_latent))
    true_rslds.dynamics.As = np.array(As)
    true_rslds.dynamics.bs = np.array(bs)
    true_rslds.dynamics.sigmasq = 1e-4 * np.ones((K, D_latent))
    
    true_rslds.transitions.Rs = Rs
    true_rslds.transitions.r = r
    
    true_rslds.emissions.inv_etas = np.log(1e-2) * np.ones((1, D_obs))
    return true_rslds

# Sample from the model
true_rslds = make_nascar_model()
z, x, y = true_rslds.sample(T=T)
print(x.shape)
print(x)

### Van der Pol System

We consider the system $$\begin{cases}\dot{x} = y\\ \dot{y} =\mu(1-x^2)y-x\end{cases}$$

#### Van der Pol single trajectory

In [ ]:
np.random.seed(1)
MU=1
IC=[0,2.2] #[y0,y_prime]


def vdp(t,y):
    dydt= np.array([y[1], MU*(1-y[0]**2)*y[1]-y[0]])
    return dydt
t_eval = np.arange(0, 200, 0.1)
sol = solve_ivp(vdp, [0, 200], IC, t_eval=t_eval)
plt.figure(figsize = (12, 4))
plt.plot(sol.y[0], sol.y[1])
plt.xlabel('t')
plt.ylabel('S(t)')
y = sol.y
y0 = np.array(y[0])
y1 = np.array(y[1])
noise0=np.random.normal(loc=0.0, scale=1.0, size=np.shape(y0))
noise1=np.random.normal(loc=0.0, scale=1.0, size=np.shape(y0))

#vdp_data = np.transpose(np.vstack((y0,y1,y0+y1+noise0,y0-y1+noise1)))
vdp_data = np.transpose(np.vstack((y0,y1,noise0,noise1)))

#### Van der Pol Multiple Trajectories

In [ ]:
np.random.seed(1)
MU = 1

def vdp(t, y):
    dydt = np.array([y[1], MU*(1-y[0]**2)*y[1]-y[0]])
    return dydt

tmax = 200

num_points = 2  # Number of initial conditions
x_values = np.linspace(-4, 4, num_points)
y_values = np.linspace(-4, 4, num_points)

initial_conditions = []

for x in x_values:
    for y in y_values:
        initial_conditions.append([x, y])

t_eval = np.arange(0, tmax, 0.1)
# initial_conditions = [[2, 0], [1, 0], [3, 0], [4,0]]  # Add more initial conditions as needed

vdp_data = []

for IC in initial_conditions:
    sol = solve_ivp(vdp, [0, tmax], IC, t_eval=t_eval)
    y0 = sol.y[0]
    y1 = sol.y[1]
    noise0 = np.random.normal(loc=0.0, scale=1.0, size=np.shape(y0))
    noise1 = np.random.normal(loc=0.0, scale=1.0, size=np.shape(y0))
    data = np.transpose(np.vstack((y0, y1, noise0, noise1)))
    vdp_data.append(data)

vdp_data = np.concatenate(vdp_data, axis=0)

plt.figure(figsize=(12, 4))
for i in range(len(initial_conditions)):
    plt.plot(vdp_data[i*t_eval.size:(i+1)*t_eval.size, 0], vdp_data[i*t_eval.size:(i+1)*t_eval.size, 1], label=f'IC: {initial_conditions[i]}')
plt.xlabel('t')
plt.ylabel('S(t)')
#plt.legend()
plt.show()

y = vdp_data[:,0:2]

### Lorenz

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp

def lorenz_deriv(t, xyz, sigma=10, beta=8/3, rho=28):
    x, y, z = xyz
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

def generate_lorenz_data(num_samples=1000, dt=0.01, sigma=10, beta=8/3, rho=28):
    t_span = np.arange(0, num_samples * dt, dt)
    initial_conditions = [-8, 7, 27]  # Adjust the initial conditions as desired
    sol = solve_ivp(lorenz_deriv, [t_span[0], t_span[-1]], initial_conditions, t_eval=t_span, args=(sigma, beta, rho))
    return sol.y.T

# Generate synthetic Lorenz system data
lorn_data = generate_lorenz_data()

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the line
ax.plot(data[:,0], data[:,1], data[:,2], color='blue')

# Customize the plot
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Line Plot')

# Show the plot
plt.show()

### Fitting Data

In [ ]:
import os
import pickle
import copy

import autograd.numpy as np
import autograd.numpy.random as npr
npr.seed(12345)

import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib inline

import seaborn as sns
color_names = ["windows blue", "red", "amber", "faded green"]
colors = sns.xkcd_palette(color_names)
sns.set_style("white")
sns.set_context("talk")

import ssm
from ssm.util import random_rotation, find_permutation

# Helper functions for plotting results
def plot_trajectory(z, x, ax=None, ls="-"):
    zcps = np.concatenate(([0], np.where(np.diff(z))[0] + 1, [z.size]))
    if ax is None:
        fig = plt.figure(figsize=(4, 4))
        ax = fig.gca()
    for start, stop in zip(zcps[:-1], zcps[1:]):
        ax.plot(x[start:stop + 1, 0],
                x[start:stop + 1, 1],
                lw=1, ls=ls,
                color=colors[z[start] % len(colors)],
                alpha=1.0)
        """
        ax.plot(x[:, 0],
                x[:, 1],
                lw=1, ls=ls,
                alpha=1.0)
        """
    return ax

def plot_observations(z, y, ax=None, ls="-", lw=1):

    zcps = np.concatenate(([0], np.where(np.diff(z))[0] + 1, [z.size]))
    if ax is None:
        fig = plt.figure(figsize=(4, 4))
        ax = fig.gca()
    T, N = y.shape
    t = np.arange(T)
    for n in range(N):
        for start, stop in zip(zcps[:-1], zcps[1:]):
            ax.plot(t[start:stop + 1], y[start:stop + 1, n],
                    lw=lw, ls=ls,
                    color=colors[z[start] % len(colors)],
                    alpha=1.0)
    return ax


def plot_most_likely_dynamics(model,
    xlim=(-4, 4), ylim=(-3, 3), nxpts=20, nypts=20,
    alpha=0.8, ax=None, figsize=(3, 3)):
    
    K = model.K
    assert model.D == 2
    x = np.linspace(*xlim, nxpts)
    y = np.linspace(*ylim, nypts)
    X, Y = np.meshgrid(x, y)
    xy = np.column_stack((X.ravel(), Y.ravel()))

    # Get the probability of each state at each xy location
    z = np.argmax(xy.dot(model.transitions.Rs.T) + model.transitions.r, axis=1)

    if ax is None:
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(111)

    for k, (A, b) in enumerate(zip(model.dynamics.As, model.dynamics.bs)):
        dxydt_m = xy.dot(A.T) + b - xy

        zk = z == k
        if zk.sum(0) > 0:
            ax.quiver(xy[zk, 0], xy[zk, 1],
                      dxydt_m[zk, 0], dxydt_m[zk, 1],
                      color=colors[k % len(colors)], alpha=alpha)

    ax.set_xlabel('$x_1$')
    ax.set_ylabel('$x_2$')

    plt.tight_layout()

    return ax

In [ ]:
# Global parameters
# T = 100
# K = 4
# D_obs = 10
# D_latent = 2
T = 2000
K = 4
D_obs = 4
D_latent = 2

#data = np.random.randn(100,10)
data = vdp_data
#data = lorn_data

In [ ]:
# Fit an rSLDS with its default initialization, using Laplace-EM with a structured variational posterior
rslds = ssm.SLDS(D_obs, K, D_latent,
             transitions="recurrent_only",
             dynamics="diagonal_gaussian",
             emissions="gaussian_orthog",
             single_subspace=True)
rslds.initialize(data)
q_elbos_lem, q_lem = rslds.fit(data, method="laplace_em",
                               variational_posterior="structured_meanfield",
                               initialize=False, num_iters=100, alpha=0.0)
xhat_lem = q_lem.mean_continuous_states[0]
# rslds.permute(find_permutation(z, rslds.most_likely_states(xhat_lem, data)))
zhat_lem = rslds.most_likely_states(xhat_lem, data)

# store rslds
rslds_lem = copy.deepcopy(rslds)

In [ ]:
# Fit an rSLDS with its default initialization, using BBVI with a structured variational posterior
rslds = ssm.SLDS(D_obs, K, D_latent, 
             transitions="recurrent_only",
             dynamics="diagonal_gaussian",
             emissions="gaussian_orthog",
             single_subspace=True)
rslds.initialize(data)

q_elbos_bbvi, q_bbvi = rslds.fit(data, method="bbvi",
                                 variational_posterior="meanfield",
                                 initialize=False, num_iters=100)

In [ ]:
# Get the posterior mean of the continuous states
xhat_bbvi = q_bbvi.mean[0]

# Find the permutation that matches the true and inferred states
#rslds.permute(find_permutation(z, rslds.most_likely_states(xhat_bbvi, y)))
#zhat_bbvi = rslds.most_likely_states(xhat_bbvi, y.T)
zhat_bbvi = rslds.most_likely_states(xhat_bbvi, vdp_data)

### Plots

In [ ]:
plt.figure(figsize=[10,8])
ax1 = plt.subplot(221)
x=y.T
z=(y1>y1.mean()).astype(int)
plot_trajectory(z, x, ax=ax1)
plt.title("True (MU=-1)(y0,y1,noise,noise)")
plt.xlabel('y0')
plt.ylabel('y1')
ax2 =plt.subplot(222)
plot_observations(z[:8000], vdp_data[:8000,:2], ax=ax2)
plt.title("Observations for first 1000 time steps")
ax3 = plt.subplot(223)
plot_trajectory(zhat_bbvi, xhat_bbvi, ax=ax3)
plt.title("Inferred, BBVI")
ax4 = plt.subplot(224)
plot_trajectory(zhat_lem, xhat_lem, ax=ax4)
plt.title("Inferred, Laplace-EM")

plt.tight_layout()


In [ ]:
"""plt.figure(figsize=(6,4))
ax = plt.subplot(111)
lim = abs(x).max(axis=0) + 1
plot_most_likely_dynamics(true_rslds, xlim=(-lim[0], lim[0]), ylim=(-lim[1], lim[1]), ax=ax)
plt.title("True Dynamics")"""

plt.figure(figsize=(6,4))
ax = plt.subplot(111)
lim = abs(xhat_lem).max(axis=0) + 1
plot_most_likely_dynamics(rslds_lem, xlim=(-lim[0], lim[0]), ylim=(-lim[1], lim[1]), ax=ax)
plt.title("Inferred Dynamics, Laplace-EM")

plt.figure(figsize=(6,4))
ax = plt.subplot(111)
lim = abs(xhat_bbvi).max(axis=0) + 1
plot_most_likely_dynamics(rslds, xlim=(-lim[0], lim[0]), ylim=(-lim[1], lim[1]), ax=ax)
plt.title("Inferred Dynamics, BBVI")